In [1]:
from bs4 import BeautifulSoup as bs
import requests
from webdriver_manager.chrome import ChromeDriverManager
from splinter import Browser
import pandas as pd
import requests

In [2]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\Rob\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


# Step 1 - Scraping
## NASA Mars News

In [3]:
url = "https://redplanetscience.com/"

browser.visit(url)

In [4]:
html = browser.html
soup = bs(html, 'html.parser')

In [5]:
print(soup.prettify())

<html>
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link crossorigin="anonymous" href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.0-beta1/dist/css/bootstrap.min.css" integrity="sha384-giJF6kkoqNQ00vy+HMDP7azOuL0xtbfIcaT9wjKHr8RbDVddVHyTfAAsrekwKmP1" rel="stylesheet"/>
  <link href="css/font.css" rel="stylesheet" type="text/css"/>
  <link href="css/app.css" rel="stylesheet" type="text/css"/>
  <link crossorigin="anonymous" href="https://pro.fontawesome.com/releases/v5.10.0/css/all.css" integrity="sha384-AYmEC3Yw5cVb3ZcuHtOA93w35dYTsvhLPVnYs9eStHfGJvOvKxVfELGroGkvsg+p" rel="stylesheet"/>
  <title>
   News - Mars Exploration Program
  </title>
 </head>
 <body>
  <div class="col-md-12">
   <div class="row">
    <nav class="navbar navbar-expand-lg navbar-light fixed-top">
     <div class="container-fluid">
      <a class="navbar-brand" href="#">
       <img src="image/nasa.png" width="80"/>
       <span class="logo">
        MA

In [6]:
news_title = soup.find_all('div', class_="content_title")


In [7]:
first_news_title = news_title[0].text
first_news_title

"NASA's InSight 'Hears' Peculiar Sounds on Mars"

In [8]:
news_paragraph = soup.find_all('div', class_="article_teaser_body")
first_news_paragraph = news_paragraph[0].text
first_news_paragraph

'Listen to the marsquakes and other, less-expected sounds that the Mars lander has been detecting.'

## JPL Mars Space Images - Featured Image

In [9]:
url = "https://spaceimages-mars.com/"

browser.visit(url)

In [10]:
html = browser.html
soup = bs(html, 'html.parser')

In [11]:
featured_image_url = browser.links.find_by_partial_href('image/featured')['href']
featured_image_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

## Mars Facts

In [5]:
url = "https://galaxyfacts-mars.com/"
header = {'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:101.0) Gecko/20100101 Firefox/101.0'}
r = requests.get(url, headers=header)
tables = pd.read_html(r.text.replace(' <br>',',').replace('<br>',','))


In [6]:
tables[0]

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 Ã 10^23 kg,5.97 Ã 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 Â°C,-88 to 58Â°C


In [14]:
tables[1]

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 Ã 10^23 kg (0.11 Earths)
3,Moons:,2 ( Phobos & Deimos )
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 Â°C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [15]:
mars_facts = tables[1].to_html()

## Mars Hemispheres

In [34]:
url = 'https://marshemispheres.com/'
browser.visit(url)

In [35]:
links = browser.links.find_by_partial_text('Hemisphere')


In [36]:
[print(links[i]['href']) for i in range(len(links))]
hemi_urls = []
[hemi_urls.append(links[i]['href']) for i in range(len(links))]

hemi_urls

https://marshemispheres.com/cerberus.html
https://marshemispheres.com/schiaparelli.html
https://marshemispheres.com/syrtis.html
https://marshemispheres.com/valles.html


['https://marshemispheres.com/cerberus.html',
 'https://marshemispheres.com/schiaparelli.html',
 'https://marshemispheres.com/syrtis.html',
 'https://marshemispheres.com/valles.html']

In [37]:
hemisphere_image_urls = []
for i in range(len(hemi_urls)):
    url = hemi_urls[i]
    browser.visit(url)
    # load image url
    link = browser.links.find_by_partial_text('Sample')[0]['href']
    
    # locate title within page
    html = browser.html
    soup = bs(html, 'html.parser')
    title = soup.find('h2', class_='title').text
    
    # append into list
    hemisphere_image_urls.append({
        'title': title,
        'img_url': link
    })
hemisphere_image_urls
    

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'}]